In [1]:
using CSV
df = CSV.read("accepted_2007_to_2018Q4.csv")
size(df)

(2260701, 151)

In [2]:
for i in names(df)
    println(i)
end

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
fico_range_low
fico_range_high
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
last_fico_range_high
last_fico_range_low
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
to

In [7]:
first(df, 5)

,Amount Requested,Application Date,Loan Title,Risk_Score
,Float64,Dates…,String⍰,Float64⍰
1,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0
2,1000.0,2007-05-26,Consolidating Debt,703.0
3,11000.0,2007-05-27,Want to consolidate my debt,715.0
4,6000.0,2007-05-27,waksman,698.0
5,1500.0,2007-05-27,mdrigo,509.0
